In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh1 = nc.Dataset(r'D:/data/SSH/SSH_2011-2015_month.nc')
data_ssh2 = nc.Dataset(r'D:/data/SSH/SSH_2016-2021_month.nc')

In [4]:
time_ssh1 = data_ssh1['time'][:]
print(len(time_ssh1))
time_ssh2 = data_ssh2['time'][:]
print(len(time_ssh2))

60
72


In [5]:
# 查看研究区域经纬度（120°E - 90°W , 0 - 60°N）
ssh_lat = data_ssh1['lat'][:].data
#print(ssh_lat[360:600])                   # 0.125 --- 59.875
ssh_lon = data_ssh1['lon'][:].data
# print(ssh_lon[720+480:])                 # 120.125 ---  179.875    240
# print(ssh_lon[:360])                     #-179.875 ---  -90.125    360

In [6]:
extend_num = 5    #输入图像大小是11，则extend_num设置为5

In [7]:
ssh1 = np.concatenate((data_ssh1['ssh'][:,360-extend_num:600+extend_num,1200-extend_num:].data,data_ssh1['ssh'][:,360-extend_num:600+extend_num,:360+extend_num].data),axis=2)
ssh2 = np.concatenate((data_ssh2['ssh'][:,360-extend_num:600+extend_num,1200-extend_num:].data,data_ssh2['ssh'][:,360-extend_num:600+extend_num,:360+extend_num].data),axis=2)

ssh =  np.concatenate((ssh1,ssh2),axis=0)

In [8]:
print(ssh.shape)
print(np.nanmax(ssh),np.nanmin(ssh))

(132, 250, 610)
2.266329 -0.19670714


In [9]:
np.sum(np.isnan(ssh[5:6,:,:]))

31583

## SSS

In [10]:
data_sss = nc.Dataset(r'D:/data/SSS/2011-2021_SSS_correct.nc')

In [11]:
# 确定时间
time_sss = data_sss['time'][:]
print(len(time_sss))

132


In [12]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[360:600])                  # 0.125 ---- 59.875
sss_lon = data_sss['lon'][:].data
#print(sss_lon[480:480+600])              # 120.125 ----- 269.875

In [13]:
sss = data_sss['sos'][:,360-extend_num:600+extend_num,480-extend_num:1080+extend_num].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 250, 610)
(132, 250, 610)


In [14]:
np.nanmax(sss),np.nanmin(sss)

(36.945, 18.755253)

In [15]:
np.sum(np.isnan(sss[5:6,:,:]))

31355

## SSW

In [16]:
data_ssw = nc.Dataset(r'D:/data/SSW/2011-2021_SSW_correct.nc')

In [17]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [18]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[360:600])      # 0.125   ---- 59.875
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[480:1080])      # 120.125  ---- 269.875

In [19]:
uwnd = data_ssw['u'][:,360-extend_num:600+extend_num,480-extend_num:1080+extend_num].data  
vwnd = data_ssw['v'][:,360-extend_num:600+extend_num,480-extend_num:1080+extend_num].data

In [20]:
print(uwnd.shape)
print(vwnd.shape)

(132, 250, 610)
(132, 250, 610)


In [21]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.203712, -13.459349, 9.599683, -14.963859)

In [22]:
print(np.sum(np.isnan(uwnd[5:6,:,:])))
print(np.sum(np.isnan(vwnd[5:6,:,:])))

0
0


## 3DT  30-2000m温度数据

In [23]:
data_3dt = nc.Dataset(r'D:/data/3DTS/2011-2021_3dt_correct.nc')

In [24]:
# 确定区域
at_3dt = data_3dt['lat'][:].data
#print(lat_3dt[329:329+240])        #0.125 --- 59.875
lon_3dt = data_3dt['lon'][:].data    
#print(lon_3dt[480:1080])            #120.125 --- 269.875

In [25]:
temp_depth = data_3dt['to'][:,:,329-extend_num:569+extend_num,480-extend_num:1080+extend_num].data

In [26]:
print(temp_depth.shape)

(132, 50, 250, 610)


In [27]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
depths_use = [50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [28]:
temp_depth_use = temp_depth[:,6:7]

In [29]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        #print(depths[i])
        temp_depth_use = np.concatenate((temp_depth_use,temp_depth[:,i:i+1]),axis=1)    

In [30]:
print(temp_depth_use.shape)
temp_depth_use = np.transpose(temp_depth_use, ( 0, 2,3,1))
print(temp_depth_use.shape)

(132, 15, 250, 610)
(132, 250, 610, 15)


In [31]:
print(np.sum(np.isnan(temp_depth_use[5:6,:,:,0:1])))
print(np.sum(np.isnan(temp_depth_use[5:6,:,:,14:])))

31350
40029


## SST表面温度数据

In [32]:
data_sst = nc.Dataset(r'D:/data/SST/SST_2011-2021_month_sheshidu_interp.nc')

In [33]:
# 确定时间
time_sst = data_sst['time'][:]
print(len(time_sst))

132


In [34]:
lat_sst = data_sst['lat'][:].data
#print(lat_sst[360:600])             #0.125 --- 59.875
lon_sst = data_sst['lon'][:].data
# print(lon_sst[720+480:])             #120.125 --- 179.875
# print(lon_sst[:360])                 #179.875 --- -90.125

In [35]:
sst = np.concatenate((data_sst['sst'][:,360-extend_num:600+extend_num,1200-extend_num:].data,data_sst['sst'][:,360-extend_num:600+extend_num,:360+extend_num].data),axis=2)

In [36]:
print(sst.shape)

(132, 250, 610)


In [37]:
np.nanmax(sst),np.nanmin(sst)

(32.69934, -1.8612671)

In [38]:
print(np.sum(np.isnan(sst[5:6,:,:])))

31574


## 温度差值

### 计算表面温度平均值与深度温度平均值

In [39]:
# 将填充值替换为nan
mean_temp = temp_depth_use
mean_temp[mean_temp == 32767] = np.nan
mean_sst = sst
mean_sst[mean_sst == 32767] = np.nan

In [40]:
# 重构数组
mean_temp = mean_temp.reshape(11,12, 250, 610, 15)    #(132, 250, 610, 15)   --->(11，12, 250, 610, 15)  
mean_sst = mean_sst.reshape(11,12,250,610)

#沿年份维度（即第一个维度）计算平均值
mean_temp = np.nanmean(mean_temp, axis=0)
print('水下温度平均值：',mean_temp.shape)

mean_sst = np.mean(mean_sst, axis=0)
print('表面温度平均值：',mean_sst.shape)

水下温度平均值： (12, 250, 610, 15)
表面温度平均值： (12, 250, 610)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_68732\2861948423.py:6: RuntimeWarning: Mean of empty slice
  mean_temp = np.nanmean(mean_temp, axis=0)


### 表面温度的平均值  减去 0-1000m温度的平均值

In [41]:
print(mean_sst.shape,mean_temp.shape)

(12, 250, 610) (12, 250, 610, 15)


In [42]:
mean_sst = np.expand_dims(mean_sst,axis=-1) #扩展维度 
mean_sst = np.repeat(mean_sst, repeats=15,axis=-1)  #(12, 250, 610)  --》(12, 250, 610，15)  
print(mean_sst.shape)

(12, 250, 610, 15)


In [43]:
mean_different = mean_sst - mean_temp
print(mean_different.shape)

(12, 250, 610, 15)


# 数据预处理

## 数据合并

In [44]:
print(ssh.shape,sss.shape,uwnd.shape,vwnd.shape,sst.shape)
cat_data =np.stack((ssh,sss,uwnd,vwnd,sst),axis=3)
cat_data.shape

(132, 250, 610) (132, 250, 610) (132, 250, 610) (132, 250, 610) (132, 250, 610)


(132, 250, 610, 5)

In [45]:
temp_depth_use.shape

(132, 250, 610, 15)

In [46]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)

In [47]:
cat_data_ss_depth.shape

(132, 250, 610, 20)

## 将两个温度异常区域的数据设置为Nan

In [48]:
temp_depth_use.shape

(132, 250, 610, 15)

In [49]:
test = temp_depth_use[:,:,:,14:15]
test[test < 0.4] = np.nan
test[test > 10] = np.nan

In [50]:
cat_data_ss_depth[:,:,:,19:] = test

In [51]:
cat_data_ss_depth.shape 

(132, 250, 610, 20)

## 将填充值赋值为Nan,且一个点中有一个维度为nan,则将这个点的所有维度都设置为nan

### 塑形，将长*宽放在前面

In [52]:
## 先对合并的数据进行reshape成（长*宽，时间*通道）
cat_data_ss_depth_reshape  = np.transpose(cat_data_ss_depth,(1,2,0,3))   # (132, 240, 600, 21) ---》 (240, 600, 132, 21)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape ((-1,132*20))  # (240, 600, 132, 21) ---》(240*600, 132*21)
print(cat_data_ss_depth_reshape.shape)

(250, 610, 132, 20)
(152500, 2640)


### 一个点中只要有一个维度为nan,则将这个点的所有维度都设置为nan 

In [53]:
# # 将填充值赋值为nan
# cat_data_ss_depth_reshape[cat_data_ss_depth_reshape == 32767] = np.nan

nan_mask = np.isnan(cat_data_ss_depth_reshape)
# nan_mask.any(axis=1)==true,只要第一个维度中含有nan就将所有设置为nan   (维度从0开始算起)
cat_data_ss_depth_reshape[nan_mask.any(axis=1) == True] = np.nan   
print('cat_data_ss_depth_reshape:',cat_data_ss_depth_reshape.shape)

cat_data_ss_depth_reshape: (152500, 2640)


### 将处理好的数据进行转化形状 

In [54]:
# 转化形状
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,132,20)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = np.transpose(cat_data_ss_depth_reshape,(1,0,2))
print(cat_data_ss_depth_reshape.shape)

(152500, 132, 20)
(132, 152500, 20)


In [55]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,250,610,20)
print(cat_data_ss_depth_reshape.shape)

(132, 250, 610, 20)


In [56]:
np.sum(np.isnan(cat_data_ss_depth_reshape[5:6,:,:,2:3]))

41910

## 将数据分成小块

In [57]:
# 表面数据
cat_data_ss = cat_data_ss_depth_reshape[:,:,:,:5]
# 深度数据
cat_data_depth = cat_data_ss_depth_reshape[:,:,:,5:]

print(cat_data_ss.shape,cat_data_depth.shape)

(132, 250, 610, 5) (132, 250, 610, 15)


In [58]:
np.sum(np.isnan(cat_data_depth[1:2,:,:,5:6]))

41910

In [59]:
# 查看差值的形状
print(mean_different.shape)

(12, 250, 610, 15)


### 移动滑块将数据处理成小块

In [60]:
# 移动滑块
import numpy as np
import pandas as pd

time_size, lat_size, lon_size, channel = cat_data_ss.shape   # （132，280，600，5）

# 定义矩形框的大小和步幅           
box_size = 11  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size) // step_size  + 1 
out_lon_size = (lon_size - box_size) // step_size  + 1 

print('time_size:',time_size,'lat_size:',lat_size,'lon_size:',lon_size)   #(132 280 600)

print(out_lat_size,out_lon_size)  #(240 600)

#输出数组 
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 240, 600, 9,9,5)

out_data_y = np.zeros((time_size, out_lat_size , out_lon_size, 15)) #(132, 272, 592,16)

out_data_lstm = np.zeros((time_size, out_lat_size , out_lon_size, 5))

out_data_mean_different_cnn = np.zeros((12, out_lat_size , out_lon_size, box_size, box_size, 15))

out_data_mean_different_lstm = np.zeros((12, out_lat_size , out_lon_size, 15))


#============================与CNN相匹配的差值===========================#
# 遍历时间步、纬度和经度
for t in range(12):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            #print(lat_min,lat_max,lon_min,lon_max)
            # 获取当前矩形框的地图数据
            box_data = mean_different[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data_mean_different_cnn[t,lat,lon, :, :, :] = box_data
            box_data = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data_mean_different_cnn的形状',out_data_mean_different_cnn.shape)
#return out_data
    


#========================CNN out_data =============================#
#遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1  , step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            #print(lat_min,lat_max,lon_min,lon_max)
            # 获取当前矩形框的地图数据
            box_data = cat_data_ss[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data的形状',out_data.shape)
#return out_data
    
#======================LSTM 与 y out_data==========================#   

# 遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(5, lat_size-5, step_size):
        for lon in range(5, lon_size-5, step_size):
            box_data_y = cat_data_depth[t, lat:lat+1, lon:lon+1,:]
            out_data_y[t,lat-5,lon-5, :] = box_data_y
            box_data_y = []
            
            box_data_lstm = cat_data_ss[t, lat:lat+1, lon:lon+1,:]
            out_data_lstm[t,lat-5,lon-5, :] = box_data_lstm
            box_data_lstm = []
            
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data_y的形状',out_data_y.shape)
print('out_data_lstm的形状',out_data_lstm.shape)   
# out_data_reshape = slider_value(cat_data)


#======================与LSTM相匹配的差值==========================#   

# 遍历时间步、纬度和经度
for t in range(12):
    sample_idx = 0
    for lat in range(5, lat_size-5, step_size):
        for lon in range(5, lon_size-5, step_size): 
            box_data_mean_different_lstm = mean_different[t, lat:lat+1, lon:lon+1,:]
            out_data_mean_different_lstm[t,lat-5,lon-5, :] = box_data_mean_different_lstm
            box_data_mean_different_lstm = []
            
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data_mean_different_lstm的形状',out_data_mean_different_lstm.shape)

# out_data_reshape = slider_value(cat_data)

time_size: 132 lat_size: 250 lon_size: 610
240 600
out_data_mean_different_cnn的形状 (12, 240, 600, 11, 11, 15)
out_data的形状 (132, 240, 600, 11, 11, 5)
out_data_y的形状 (132, 240, 600, 15)
out_data_lstm的形状 (132, 240, 600, 5)
out_data_mean_different_lstm的形状 (12, 240, 600, 15)


## 删除缺失值多于一半的数据

### 获取Nan_mask

In [61]:
# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,11,11,5))
print(out_data_reshape.shape) 
# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

(132, 144000, 11, 11, 5)
(132, 144000, 11, 11, 5)


In [62]:
# （137344，9，9）中存在4500416个nan
np.count_nonzero(nan_mask_out_data[3:4,:,:,:,1:2])

4500416

In [63]:
# 选取一个作为代表
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,0].reshape(-1,11*11) #161024, 9, 9) -->（161024，81）    # 其中true 代表Nan

In [64]:
# # 查看mask中Nan 的个数
# print(np.count_nonzero(nan_mask_out_data_one))    # 2724451
# #查看形状
# print(nan_mask_out_data_one.shape)     # (137344, 81)
# # 查看内容
# nan_mask_out_data_one

### 为有效值大于一半的方块标注True

In [65]:
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=61):    #True 代表nan，当nan小于等于61时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

In [66]:
np.count_nonzero(nan_mask)  # 查看有效值个数

107191

### 标注lstm中的缺失值

In [67]:
## 标注lstm中的缺失值
out_data_lstm_reshape = out_data_lstm.reshape((132,-1,5))
print(out_data_lstm_reshape.shape)
# 获取变量 nan 掩码
nan_mask_lstm = np.isnan(out_data_lstm_reshape)
print(nan_mask_lstm.shape)

(132, 144000, 5)
(132, 144000, 5)


In [68]:
np.count_nonzero(~nan_mask_lstm[1,:,4])  

106839

### 合并缺失值

In [69]:
nan_mask_cat = [a and b for a, b in zip(nan_mask, ~nan_mask_lstm[0,:,0])]

In [70]:
np.count_nonzero(nan_mask_cat)  

106512

In [71]:
np.save("../data_test/nan_mask_cat.npy",nan_mask_cat)

### 修改数据形状，使其可以通过nan_mask_cat删除无效数据

In [72]:
out_data_mean_different_lstm.shape ,out_data_lstm.shape

((12, 240, 600, 15), (132, 240, 600, 5))

In [73]:
out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)

out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,15)
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_lstm_reshape =  np.transpose(out_data_lstm,(1,2,0,3))
print('out_data_lstm_reshape:',out_data_lstm_reshape.shape)
out_data_lstm_reshape = out_data_lstm_reshape.reshape(-1,132,5)
print('out_data_lstm_reshape:',out_data_lstm_reshape.shape) 

#out_data_mean_different_cnn = np.array(out_data_mean_different_cnn)
out_data_mean_different_cnn_reshape =  np.transpose(out_data_mean_different_cnn,(1,2,0,3,4,5))
print('out_data_mean_different_cnn_reshape:',out_data_mean_different_cnn_reshape.shape)
out_data_mean_different_cnn_reshape = out_data_mean_different_cnn_reshape.reshape(-1,12,11,11,15)
print('out_data_mean_different_cnn_reshape:',out_data_mean_different_cnn_reshape.shape) 

out_data_mean_different_lstm_reshape =  np.transpose(out_data_mean_different_lstm,(1,2,0,3))
print('out_data_mean_different_lstm_reshape:',out_data_mean_different_lstm_reshape.shape)
out_data_mean_different_lstm_reshape = out_data_mean_different_lstm_reshape.reshape(-1,12,15)
print('out_data_mean_different_lstm_reshape:',out_data_mean_different_lstm_reshape.shape) 


out_data_reshape: (144000, 132, 11, 11, 5)
out_data_y_reshape: (240, 600, 132, 15)
out_data_y_reshape: (144000, 132, 15)
out_data_lstm_reshape: (240, 600, 132, 5)
out_data_lstm_reshape: (144000, 132, 5)
out_data_mean_different_cnn_reshape: (240, 600, 12, 11, 11, 15)
out_data_mean_different_cnn_reshape: (144000, 12, 11, 11, 15)
out_data_mean_different_lstm_reshape: (240, 600, 12, 15)
out_data_mean_different_lstm_reshape: (144000, 12, 15)


### 开始删除无效数据

In [74]:
out_data_y_reshape.shape

(144000, 132, 15)

In [75]:
out_data_reshape = out_data_reshape[nan_mask_cat]
out_data_y_reshape = out_data_y_reshape[nan_mask_cat]
out_data_lstm_reshape = out_data_lstm_reshape[nan_mask_cat]
out_data_mean_different_cnn_reshape = out_data_mean_different_cnn_reshape[nan_mask_cat]
out_data_mean_different_lstm_reshape = out_data_mean_different_lstm_reshape[nan_mask_cat]

In [76]:
print(out_data_reshape.shape,out_data_y_reshape.shape,out_data_lstm_reshape.shape,
      out_data_mean_different_cnn_reshape.shape,out_data_mean_different_lstm_reshape.shape)

(106512, 132, 11, 11, 5) (106512, 132, 15) (106512, 132, 5) (106512, 12, 11, 11, 15) (106512, 12, 15)


### 使用0来填充Nan

In [77]:
out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103952, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103952, 132, 15)
out_data_lstm_reshape = np.nan_to_num(out_data_lstm_reshape, nan=0)
out_data_mean_different_cnn_reshape = np.nan_to_num(out_data_mean_different_cnn_reshape, nan=0)
out_data_mean_different_lstm_reshape = np.nan_to_num(out_data_mean_different_lstm_reshape, nan=0)

# 验证集和测试集的划分

## 对数据进行reshape,以进行数据集的划分

In [78]:
# print(out_data_mean_different_reshape.shape,mean_different.shape)

In [79]:
# 将时间放到前面
out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_lstm_reshape =  np.transpose(out_data_lstm_reshape,(1,0,2))
print('out_data_lstm_reshape:',out_data_lstm_reshape.shape)

out_data_mean_different_cnn_reshape = np.transpose(out_data_mean_different_cnn_reshape,(1,0,2,3,4))
print('out_data_mean_different_cnn_reshape',out_data_mean_different_cnn_reshape.shape)
out_data_mean_different_lstm_reshape = np.transpose(out_data_mean_different_lstm_reshape,(1,0,2))
print('out_data_mean_different_lstm_reshape',out_data_mean_different_lstm_reshape.shape)

out_data_reshape: (132, 106512, 11, 11, 5)
out_data_y_reshape: (132, 106512, 15)
out_data_lstm_reshape: (132, 106512, 5)
out_data_mean_different_cnn_reshape (12, 106512, 11, 11, 15)
out_data_mean_different_lstm_reshape (12, 106512, 15)


In [80]:
# np.save("../data_test/out_data_reshape.npy",out_data_reshape)
# np.save("../data_test/out_data_y_reshape.npy",out_data_y_reshape)
# np.save("../data_test/out_data_lstm_reshape.npy",out_data_lstm_reshape)
# np.save("../data_test/out_data_mean_different_cnn_reshape.npy",out_data_mean_different_cnn_reshape)
# np.save("../data_test/out_data_mean_different_lstm_reshape.npy",out_data_mean_different_lstm_reshape)

### 使用每个表面数据减去平均差值

In [81]:
'''当前运行的是30-150m的代码 因此写的是out_data_mean_different_cnn_reshape[:,:,:,:,:5]， 200-500m就是5:10 ,600-1000m就是10:'''

out_data_mean_different_cnn_reshape_1 = np.expand_dims(out_data_mean_different_cnn_reshape[:,:,:,:,:5], axis=0)
print(out_data_mean_different_cnn_reshape_1.shape) # 扩展第一个维度    (1, 12, 106512, 11, 11, 5)
out_data_mean_different_cnn_reshape_1 = np.repeat(out_data_mean_different_cnn_reshape_1, repeats=11, axis=0)
print(out_data_mean_different_cnn_reshape_1.shape) # 将第一个维度重复11次    (11, 12, 106512, 11, 11, 5)
out_data_mean_different_cnn_reshape_1 = out_data_mean_different_cnn_reshape_1.reshape(-1,106512, 11, 11, 5)
print(out_data_mean_different_cnn_reshape_1.shape) #将前两个维度相乘    (132, 106512, 11, 11, 5)

(1, 12, 106512, 11, 11, 5)
(11, 12, 106512, 11, 11, 5)
(132, 106512, 11, 11, 5)


In [82]:
# 使用每个表面数据减去这个差值,得到最终结果。out_data_reshape[:,:,:,:,4:]代表sst
end_cnn_different = (np.repeat(out_data_reshape[:,:,:,:,4:], repeats=5, axis=-1) 
                     -out_data_mean_different_cnn_reshape_1) 

In [83]:
#np.save("../data_test/end_cnn_different.npy",end_cnn_different)

In [84]:
'''当前运行的是30-150m的代码 因此写的是out_data_mean_different_lstm_reshape[:,:,:,:,:5]， 200-500m就是5:10 ,600-1000m就是10:'''

out_data_mean_different_lstm_reshape_1 = np.expand_dims(out_data_mean_different_lstm_reshape[:,:,:5], axis=0)
print(out_data_mean_different_lstm_reshape_1.shape)
out_data_mean_different_lstm_reshape_1 = np.repeat(out_data_mean_different_lstm_reshape_1, repeats=11, axis=0)
print(out_data_mean_different_lstm_reshape_1.shape)
out_data_mean_different_lstm_reshape_1 = out_data_mean_different_lstm_reshape_1.reshape(-1,106512, 5)
print(out_data_mean_different_lstm_reshape_1.shape)

(1, 12, 106512, 5)
(11, 12, 106512, 5)
(132, 106512, 5)


In [85]:
# 使用每个表面数据减去这个差值,得到最终结果。out_data_lstm_reshape[:,:,4:]代表sst
end_lstm_different = (np.repeat(out_data_lstm_reshape[:,:,4:], repeats=5, axis=-1) 
                     -out_data_mean_different_lstm_reshape_1) 

## 将数据进行堆叠

In [86]:
available = len(out_data_lstm_reshape) 
predict = 1

out_data_lstm_reshape_shape = out_data_lstm_reshape.shape[1:]
N_stack = 10
X_len = available - (N_stack-1)

X_lstm = np.zeros(shape = (X_len, N_stack,out_data_lstm_reshape_shape[0],out_data_lstm_reshape_shape[1]))  
for i in range(X_len):
    X_lstm[i] = np.stack(out_data_lstm_reshape[i:i+N_stack])
    #print(i)

print(X_lstm.shape)
#print(Y.shape)

(123, 10, 106512, 5)


In [87]:
# 对end_lstm_different进行堆叠

available = len(end_lstm_different) 
predict = 1

end_lstm_different_shape = end_lstm_different.shape[1:]  #(106512,16)

N_stack = 10
X_len = available - (N_stack-1)

X_lstm_different = np.zeros(shape = (X_len, N_stack,end_lstm_different_shape[0],end_lstm_different_shape[1]))  

for i in range(X_len):
    X_lstm_different[i] = np.stack(end_lstm_different[i:i+N_stack])
    #print(i)

print(X_lstm_different.shape)
#print(Y.shape)

(123, 10, 106512, 5)


## 统一时间

In [88]:
Y = out_data_y_reshape[9:]
X_cnn = out_data_reshape[9:]
X_cnn_different = end_cnn_different[9:]
print(X_lstm.shape,Y.shape,X_cnn.shape,X_cnn_different.shape,X_lstm_different.shape)

(123, 10, 106512, 5) (123, 106512, 15) (123, 106512, 11, 11, 5) (123, 106512, 11, 11, 5) (123, 10, 106512, 5)


## 将差值合并

In [89]:
X_cnn = np.concatenate((X_cnn,X_cnn_different),axis=-1)      # 表面数据和差值合并为X_cnn
X_lstm = np.concatenate((X_lstm,X_lstm_different),axis=-1)

print('X_cnn.shape:',X_cnn.shape,'X_lstm.shape:',X_lstm.shape)

X_cnn.shape: (123, 106512, 11, 11, 10) X_lstm.shape: (123, 10, 106512, 10)


In [3]:
'''
当前运行的是30-150m的代码 因此写的是X_cnn_30_150m.npy， 
200-500m就是X_cnn_200_500m.npy ,
600-1000m就是X_cnn_600_1000m.npy'''

# np.save("../data_test/xy/X_cnn_30_150m.npy",X_cnn)
# np.save("../data_test/xy/X_lstm_30_150m.npy",X_lstm)
# np.save("../data_test/xy/Y_30_150m.npy",Y)

#30,50,100,125,150
X_cnn = np.load("../data_test/xy/X_cnn_30_150m.npy")
X_lstm = np.load("../data_test/xy/X_lstm_30_150m.npy")
Y = np.load("../data_test/xy/Y_30_150m.npy")

In [4]:
# Y.shape 
# ## 获取Y的平均值
# Y_mean = np.nanmean(Y,axis=0)
# Y_mean.shape 
# Y_anom = Y - Y_mean
# Y =  Y_anom

In [5]:
# # 获取当前 Jupyter 环境中的所有全局变量
# all_vars = list(globals().keys())

# # 要保留的变量列表
# keep_vars = ['X_cnn', 'X_lstm', 'Y']

# # 删除除 keep_vars 中指定的变量以外的所有变量
# for var in all_vars:
#     if var not in keep_vars and not var.startswith('_'):
#         del globals()[var]

## 划分数据集

In [6]:
# 最后十二个月的数据作为test
test_num = 111
x_cnn_train = X_cnn[:test_num]
x_lstm_train = X_lstm[:test_num]
y_train = Y[:test_num]

x_cnn_test = X_cnn[test_num:]
x_lstm_test = X_lstm[test_num:]
y_test = Y[test_num:]

print(x_cnn_train.shape,x_lstm_train.shape,y_train.shape,x_cnn_test.shape,x_lstm_test.shape,y_test.shape)

(111, 106512, 11, 11, 10) (111, 10, 106512, 10) (111, 106512, 15) (12, 106512, 11, 11, 10) (12, 10, 106512, 10) (12, 106512, 15)


In [7]:
# 将前两个维度进行合并  时间*(lat*lon) = batchsize
x_cnn_train = np.reshape(x_cnn_train,(-1,11,11,10))
x_cnn_test = np.reshape(x_cnn_test,(-1,11,11,10))

y_train = np.reshape(y_train,(-1,15))
y_test = np.reshape(y_test,(-1,15))

In [8]:
x_lstm_train = np.transpose(x_lstm_train, (0,2,1,3))  # (115, 6, 103623, 5)  ---> (115, 103623, 6, 5)
x_lstm_train = x_lstm_train.reshape((-1,10,10))          #(115, 103623, 6, 5)  ----->(115*103623, 6, 5)
x_lstm_test = np.transpose(x_lstm_test, (0,2,1,3))
x_lstm_test = x_lstm_test.reshape((-1,10,10))

In [9]:
print('x_lstm_train',x_lstm_train.shape)
print('x_cnn_train',x_cnn_train.shape)
print('y_train',y_train.shape)

print('x_cnn_test:',x_cnn_test.shape)
print('x_lstm_test:',x_lstm_test.shape)
print('y_test',y_test.shape)

x_lstm_train (11822832, 10, 10)
x_cnn_train (11822832, 11, 11, 10)
y_train (11822832, 15)
x_cnn_test: (1278144, 11, 11, 10)
x_lstm_test: (1278144, 10, 10)
y_test (1278144, 15)


## 划分训练数据和目标数据

In [10]:
x_cnn_train_split, x_cnn_val, y_train_split, y_val = train_test_split(x_cnn_train , y_train, test_size=0.3, random_state=100)
x_lstm_train_split, x_lstm_val, y_train_split_2, y_val_2 = train_test_split(x_lstm_train, y_train, test_size=0.3, random_state=100)

In [11]:
# 输出形状
print('x_cnn_train_split',x_cnn_train_split.shape)
print('x_lstm_train_split',x_lstm_train_split.shape)
print('y_train_split',y_train_split.shape)

print('x_cnn_test',x_cnn_test.shape)
print('x_lstm_test',x_lstm_test.shape)
print('y_test',y_test.shape)

print('x_cnn_val',x_cnn_val.shape)
print('x_lstm_val',x_lstm_val.shape)
print('y_val',y_val.shape)

x_cnn_train_split (8275982, 11, 11, 10)
x_lstm_train_split (8275982, 10, 10)
y_train_split (8275982, 15)
x_cnn_test (1278144, 11, 11, 10)
x_lstm_test (1278144, 10, 10)
y_test (1278144, 15)
x_cnn_val (3546850, 11, 11, 10)
x_lstm_val (3546850, 10, 10)
y_val (3546850, 15)


# 特征归一化

In [12]:
# 记录数据集的形状
# train 
x_cnn_train_split_shape = x_cnn_train_split.shape
x_lstm_train_split_shape = x_lstm_train_split.shape 
y_train_split_shape  = y_train_split.shape

# test 
x_cnn_test_shape = x_cnn_test.shape
x_lstm_test_shape = x_lstm_test.shape
y_test_shape = y_test.shape

#val
x_cnn_val_shape = x_cnn_val.shape
x_lstm_val_shape = x_lstm_val.shape
y_val_shape = y_val.shape

In [13]:
# 变量标准化
scaler_f = StandardScaler()
scaler_f_2 = StandardScaler()
x_cnn_train_split = scaler_f.fit_transform(x_cnn_train_split.reshape(-1,x_cnn_train_split_shape[3])) 
x_lstm_train_split = scaler_f_2.fit_transform(x_lstm_train_split.reshape(-1,x_lstm_train_split_shape[2])) 

x_cnn_test = scaler_f.transform(x_cnn_test.reshape(-1,x_cnn_test_shape[3]))
x_lstm_test = scaler_f_2.transform(x_lstm_test.reshape(-1,x_lstm_test_shape[2]))

x_cnn_val = scaler_f.transform(x_cnn_val.reshape(-1,x_cnn_val_shape[3]))
x_lstm_val = scaler_f_2.transform(x_lstm_val.reshape(-1,x_lstm_val_shape[2]))

In [14]:
# 目标值标准化
scaler_l = StandardScaler()
y_train_split = scaler_l.fit_transform(y_train_split.reshape(-1,y_train_split_shape[1])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[1])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[1])) 

In [15]:
# reshape
x_cnn_train_split = np.reshape(x_cnn_train_split,(-1,11,11,10))
x_cnn_test = np.reshape(x_cnn_test,(-1,11,11,10))
x_cnn_val = np.reshape(x_cnn_val,(-1,11,11,10))


x_lstm_train_split = np.reshape(x_lstm_train_split,(-1,10,10))
x_lstm_test = np.reshape(x_lstm_test,(-1,10,10))
x_lstm_val = np.reshape(x_lstm_val,(-1,10,10))


y_train_split = np.reshape(y_train_split,(-1,15))
y_test = np.reshape(y_test,(-1,15))
y_val = np.reshape(y_val,(-1,15))

In [16]:
# 输出形状
print('x_cnn_train_split',x_cnn_train_split.shape)
print('x_lstm_train_split',x_lstm_train_split.shape)
print('y_train_split',y_train_split.shape)

print('x_cnn_test',x_cnn_test.shape)
print('x_lstm_test',x_lstm_test.shape)
print('y_test',y_test.shape)

print('x_cnn_val',x_cnn_val.shape)
print('x_lstm_val',x_lstm_val.shape)
print('y_val',y_val.shape)

x_cnn_train_split (8275982, 11, 11, 10)
x_lstm_train_split (8275982, 10, 10)
y_train_split (8275982, 15)
x_cnn_test (1278144, 11, 11, 10)
x_lstm_test (1278144, 10, 10)
y_test (1278144, 15)
x_cnn_val (3546850, 11, 11, 10)
x_lstm_val (3546850, 10, 10)
y_val (3546850, 15)


# 定义网络结构

## 卷积注意力

In [17]:
# arr = np.random.rand(2,5, 5, 6)   
# arr2 = np.random.rand(2,1, 1, 6)  
# a = arr * arr2
# print(a.shape)

In [ ]:
# 卷积注意力
def ChannelAttention(inputs,in_planes, ratio=2):
    
    avg_out= layers.GlobalAveragePooling2D()
    max_out= layers.GlobalMaxPooling2D()

    fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            activation=tf.nn.relu,
                            use_bias=True, bias_initializer='zeros')
    fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            use_bias=True, bias_initializer='zeros')

    
    avg_out = avg_out(inputs)
    max_out = max_out(inputs)
    out = tf.stack([avg_out, max_out], axis=1)  # shape=(None, 2, fea_num)
    out = fc2(fc1(out))
    out = tf.reduce_sum(out, axis=1)             # shape=(256, 512)
    out = tf.nn.sigmoid(out)
    out = layers.Reshape((1, 1, out.shape[1]))(out)

    return  inputs * out

def regularized_padded_conv(*args, **kwargs):
    return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(5e-4))

def SpatialAttention(inputs,kernel_size=5):
    conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
    avg_out = tf.reduce_mean(inputs, axis=3)
    max_out = tf.reduce_max(inputs, axis=3)
    out = tf.stack([avg_out, max_out], axis=-1)             # 创建一个维度,拼接到一起concat。
    out = conv1(out)
    return inputs * out


def cbam_block(inputs, in_planes,ratio=2):

    cbam_feature = ChannelAttention(inputs, in_planes,ratio)
    cbam_feature = SpatialAttention(cbam_feature)
    return cbam_feature

## 一对多注意力

In [ ]:
#  一对多注意力
# 注意力层的定制实现，以适应多次注意力调用
class Attention(Layer):
    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)
        
    def __call__(self, inputs):
        """
        Keras的多对一注意力机制。
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        """
        
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])  
        
        # Inside dense layer 内部全连接层
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W是可训练权重矩阵 
        
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states) 
        # 进行一次全连接，本质上没有太大意义，只是为了增加一下模型复杂度，
        # 并且改变一下前T个时间步的特征表示,这里其实也可以不进行全连接，直接使用hidden_states
        
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        # # 注意力，增加对最后一个时间步长的关注
        
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)  # 取出最后一个时间步的特征
        score = Dot(axes=[1, 2])([h_t, score_first_part])  # 将最后一个时间步长与前T个时间步长进行点积
        attention_weights = Activation('softmax')(score)   # 求权重（将权值通过softmax进行归一化，得到直接可用的权重）
        
        
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1])([hidden_states, attention_weights])  # 上下文向量
        pre_activation = Concatenate()([context_vector, h_t]) #再次将最后一个时间步的信息加入，本质上是一个残差连接，这步可以省略
        attention_vector = Dense(self.units, use_bias=False, activation='tanh')(pre_activation) # 用一个全连接层进行合并
        
        print('注意力被执行了一次')
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

## 模型

In [27]:
def create_model():
    # =========================================== CNN + CBAM ======================================== #
    model_input= Input(shape=( 11, 11, 6))
    output_size = 1
    
    x = cbam_block(model_input,6)
    
    x = Conv2D(filters=32, kernel_size=(3, 3),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,32)
    
    x = Conv2D(filters=64, kernel_size=(3, 3),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,64)
    
    x = Flatten()(x)
    x = Dense(units=32,activation = 'relu')(x)
    out = Dense(units=16,activation = 'relu')(x)
    out = Dense(units=1)(x)
    
    # =========================================== LSTM =============================================== #
    #MHA = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=4)
    
    model2_input= Input(shape=(10,6))
    
    model2 = tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True))(model2_input)   
    model2 = Dropout(0.2)(model2)
    
    model2 = tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True))(model2)

    model2 = Attention()(model2)
    
    model2 = Flatten()(model2)
    
    out2 = Dense(1)(model2)
    # =========================================== LSTM 2=============================================== #
    
    model3_input= Input(shape=(3,5))
   
    model3= tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True))(model3_input)   
    model3 = Dropout(0.2)(model3)
    
    model3 = tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True))(model3)
    model3 = Attention()(model3)
    
    model3 = Flatten()(model3)
    
    out3 = Dense(1)(model3)

    
    # ============================================ 合并 ========================================= #
    ensemble = Concatenate()([out, out2])  
    #ensemble=Dense(64)(ensemble) 
    ensemble=Dense(16)(ensemble) 
    merged_model=Dense(1)(ensemble) 
    
    model = Model(inputs=[model_input,model2_input], outputs=merged_model)
    #model = Model(inputs=model_input, outputs=out)
    
    #model.summary()
    #model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model 

In [28]:
model = create_model()
#model.summary()

注意力被执行了一次
注意力被执行了一次


# 模型训练

## 对指定层进行训练

In [29]:
# initial_learning_rate = 0.0005
# optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)


model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_path='../model/cnn_lstm_jc_newVariable/anom/cnn_lstm_jc_model_newVariable__1000m.h5'
keras_callbacks   = [
      EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
]

In [21]:
input_cnn_train = np.concatenate((x_cnn_train_split[:,:,:,:5],x_cnn_train_split[:,:,:,9:]),axis=-1) 
input_lstm_train = np.concatenate((x_lstm_train_split[:,:,:5],x_lstm_train_split[:,:,9:]),axis=-1)
print(input_cnn_train.shape)
print(input_lstm_train.shape)

input_cnn_val = np.concatenate((x_cnn_val[:,:,:,:5],x_cnn_val[:,:,:,9:]),axis=-1)  
input_lstm_val = np.concatenate((x_lstm_val[:,:,:5],x_lstm_val[:,:,9:]),axis=-1)  
print(input_cnn_val.shape)
print(input_lstm_val.shape)

(8328740, 11, 11, 6)
(8328740, 10, 6)
(3569461, 11, 11, 6)
(3569461, 10, 6)


In [24]:
num_epochs = 1000
batch_size = 10240


history = model.fit([input_cnn_train,input_lstm_train], y_train_split[:,14:15],
                    validation_data=([input_cnn_val,input_lstm_val],y_val[:,14:15]),
                    epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

Epoch 1/1000
814/814 - 115s - loss: 0.0805 - val_loss: 0.0245
Epoch 2/1000
814/814 - 84s - loss: 0.0193 - val_loss: 0.0152
Epoch 3/1000
814/814 - 84s - loss: 0.0144 - val_loss: 0.0127
Epoch 4/1000
814/814 - 84s - loss: 0.0125 - val_loss: 0.0116
Epoch 5/1000
814/814 - 84s - loss: 0.0112 - val_loss: 0.0100
Epoch 6/1000
814/814 - 84s - loss: 0.0103 - val_loss: 0.0097
Epoch 7/1000
814/814 - 84s - loss: 0.0096 - val_loss: 0.0089
Epoch 8/1000
814/814 - 84s - loss: 0.0092 - val_loss: 0.0087
Epoch 9/1000
814/814 - 84s - loss: 0.0088 - val_loss: 0.0082
Epoch 10/1000
814/814 - 84s - loss: 0.0085 - val_loss: 0.0077
Epoch 11/1000
814/814 - 84s - loss: 0.0083 - val_loss: 0.0075
Epoch 12/1000
814/814 - 85s - loss: 0.0080 - val_loss: 0.0075
Epoch 13/1000
814/814 - 84s - loss: 0.0078 - val_loss: 0.0071
Epoch 14/1000
814/814 - 84s - loss: 0.0076 - val_loss: 0.0071
Epoch 15/1000
814/814 - 84s - loss: 0.0075 - val_loss: 0.0068
Epoch 16/1000
814/814 - 83s - loss: 0.0073 - val_loss: 0.0067
Epoch 17/1000
81

## 对所有层进行训练

In [30]:
def train_model(x_train1,x_train2,y_train,x_val1,x_val2,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    
    model.fit([x_train1,x_train2], y_train, 
        validation_data=([x_val1,x_val2],y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [31]:
def train_cnn_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    
    model.fit(x_train, y_train, 
        validation_data=(x_val,y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [32]:
def train_lstm_model(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(optimizer='adam', loss='mean_squared_error')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    
    model.fit(x_train, y_train, 
        validation_data=(x_val,y_val),
        epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [23]:
# input_cnn_train = np.concatenate((x_cnn_train_split[:,:,:,:5],x_cnn_train_split[:,:,:,9:]),axis=-1) 
# input_lstm_train = np.concatenate((x_lstm_train_split[:,:,:5],x_lstm_train_split[:,:,9:]),axis=-1)
# print(input_cnn_train.shape)
# print(input_lstm_train.shape)

# input_cnn_val = np.concatenate((x_cnn_val[:,:,:,:5],x_cnn_val[:,:,:,9:]),axis=-1)  
# input_lstm_val = np.concatenate((x_lstm_val[:,:,:5],x_lstm_val[:,:,9:]),axis=-1)  
# print(input_cnn_val.shape)
# print(input_lstm_val.shape)

In [33]:
y_train_split.shape

(8275982, 15)

### 600-1000m

In [25]:
#depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
depth = [600,700,800,900,1000]
for i in range(0,5):  #i = 0---14
    input_cnn_train = np.concatenate((x_cnn_train_split[:,:,:,:5],x_cnn_train_split[:,:,:,i+5:i+6]),axis=-1) 
    input_lstm_train = np.concatenate((x_lstm_train_split[:,:,:5],x_lstm_train_split[:,:,i+5:i+6]),axis=-1)
    input_cnn_val = np.concatenate((x_cnn_val[:,:,:,:5],x_cnn_val[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_val = np.concatenate((x_lstm_val[:,:,:5],x_lstm_val[:,:,i+5:i+6]),axis=-1)  
    
    train_model(input_cnn_train,input_lstm_train,y_train_split[:,i+10:i+11],
                input_cnn_val,input_lstm_val,y_val[:,i+10:i+11],
                '../model/cnn_lstm_jc_model_newVariable4_'+str(depth[i])+'m.h5')

注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 112s - loss: 0.1111 - val_loss: 0.0514
Epoch 2/1000
809/809 - 82s - loss: 0.0352 - val_loss: 0.0265
Epoch 3/1000
809/809 - 82s - loss: 0.0217 - val_loss: 0.0180
Epoch 4/1000
809/809 - 82s - loss: 0.0172 - val_loss: 0.0180
Epoch 5/1000
809/809 - 82s - loss: 0.0152 - val_loss: 0.0170
Epoch 6/1000
809/809 - 82s - loss: 0.0140 - val_loss: 0.0130
Epoch 7/1000
809/809 - 82s - loss: 0.0132 - val_loss: 0.0128
Epoch 8/1000
809/809 - 82s - loss: 0.0124 - val_loss: 0.0116
Epoch 9/1000
809/809 - 82s - loss: 0.0118 - val_loss: 0.0121
Epoch 10/1000
809/809 - 82s - loss: 0.0114 - val_loss: 0.0105
Epoch 11/1000
809/809 - 82s - loss: 0.0109 - val_loss: 0.0098
Epoch 12/1000
809/809 - 82s - loss: 0.0106 - val_loss: 0.0097
Epoch 13/1000
809/809 - 81s - loss: 0.0103 - val_loss: 0.0093
Epoch 14/1000
809/809 - 81s - loss: 0.0100 - val_loss: 0.0088
Epoch 15/1000
809/809 - 81s - loss: 0.0097 - val_loss: 0.0087
Epoch 16/1000
809/809 - 81s - loss: 0.0095 - val_loss: 0.0

Epoch 64/1000
809/809 - 80s - loss: 0.0061 - val_loss: 0.0050
Epoch 65/1000
809/809 - 80s - loss: 0.0060 - val_loss: 0.0051
Epoch 66/1000
809/809 - 80s - loss: 0.0060 - val_loss: 0.0051
Epoch 67/1000
809/809 - 80s - loss: 0.0060 - val_loss: 0.0052
Epoch 68/1000
809/809 - 80s - loss: 0.0059 - val_loss: 0.0051
Epoch 69/1000
809/809 - 80s - loss: 0.0059 - val_loss: 0.0051
Epoch 70/1000
809/809 - 80s - loss: 0.0059 - val_loss: 0.0050
Epoch 71/1000
809/809 - 80s - loss: 0.0059 - val_loss: 0.0051
Epoch 72/1000
809/809 - 80s - loss: 0.0059 - val_loss: 0.0051
Epoch 73/1000
809/809 - 80s - loss: 0.0058 - val_loss: 0.0050
注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 91s - loss: 0.0796 - val_loss: 0.0278
Epoch 2/1000
809/809 - 77s - loss: 0.0221 - val_loss: 0.0185
Epoch 3/1000
809/809 - 78s - loss: 0.0172 - val_loss: 0.0155
Epoch 4/1000
809/809 - 78s - loss: 0.0153 - val_loss: 0.0144
Epoch 5/1000
809/809 - 78s - loss: 0.0141 - val_loss: 0.0127
Epoch 6/1000
809/809 - 78s - loss: 0.0130 - val_loss: 0

Epoch 50/1000
809/809 - 79s - loss: 0.0061 - val_loss: 0.0053
Epoch 51/1000
809/809 - 79s - loss: 0.0061 - val_loss: 0.0052
Epoch 52/1000
809/809 - 79s - loss: 0.0061 - val_loss: 0.0052
Epoch 53/1000
809/809 - 79s - loss: 0.0061 - val_loss: 0.0053
Epoch 54/1000
809/809 - 79s - loss: 0.0060 - val_loss: 0.0052
Epoch 55/1000
809/809 - 79s - loss: 0.0060 - val_loss: 0.0055
Epoch 56/1000
809/809 - 79s - loss: 0.0060 - val_loss: 0.0051
Epoch 57/1000
809/809 - 79s - loss: 0.0059 - val_loss: 0.0051
Epoch 58/1000
809/809 - 79s - loss: 0.0059 - val_loss: 0.0050
Epoch 59/1000
809/809 - 79s - loss: 0.0059 - val_loss: 0.0054
Epoch 60/1000
809/809 - 79s - loss: 0.0059 - val_loss: 0.0050
Epoch 61/1000
809/809 - 79s - loss: 0.0059 - val_loss: 0.0053
Epoch 62/1000
809/809 - 79s - loss: 0.0058 - val_loss: 0.0050
Epoch 63/1000
809/809 - 79s - loss: 0.0058 - val_loss: 0.0049
Epoch 64/1000
809/809 - 79s - loss: 0.0058 - val_loss: 0.0050
Epoch 65/1000
809/809 - 79s - loss: 0.0058 - val_loss: 0.0049
Epoch 66

### 200m-500m

In [21]:
#depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
depth = [200,250,300,400,500]
for i in range(0,5): 
    input_cnn_train = np.concatenate((x_cnn_train_split[:,:,:,:5],x_cnn_train_split[:,:,:,i+5:i+6]),axis=-1) 
    input_lstm_train = np.concatenate((x_lstm_train_split[:,:,:5],x_lstm_train_split[:,:,i+5:i+6]),axis=-1)
    input_cnn_val = np.concatenate((x_cnn_val[:,:,:,:5],x_cnn_val[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_val = np.concatenate((x_lstm_val[:,:,:5],x_lstm_val[:,:,i+5:i+6]),axis=-1)  
    
    train_model(input_cnn_train,input_lstm_train,y_train_split[:,i+5:i+6],
                input_cnn_val,input_lstm_val,y_val[:,i+5:i+6],
                '../model/cnn_lstm_jc_model_newVariable3_'+str(depth[i])+'m.h5')

注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 113s - loss: 0.0961 - val_loss: 0.0423
Epoch 2/1000
809/809 - 83s - loss: 0.0259 - val_loss: 0.0168
Epoch 3/1000
809/809 - 83s - loss: 0.0133 - val_loss: 0.0112
Epoch 4/1000
809/809 - 83s - loss: 0.0099 - val_loss: 0.0088
Epoch 5/1000
809/809 - 83s - loss: 0.0087 - val_loss: 0.0083
Epoch 6/1000
809/809 - 82s - loss: 0.0081 - val_loss: 0.0076
Epoch 7/1000
809/809 - 81s - loss: 0.0076 - val_loss: 0.0067
Epoch 8/1000
809/809 - 81s - loss: 0.0073 - val_loss: 0.0068
Epoch 9/1000
809/809 - 81s - loss: 0.0069 - val_loss: 0.0062
Epoch 10/1000
809/809 - 81s - loss: 0.0066 - val_loss: 0.0059
Epoch 11/1000
809/809 - 81s - loss: 0.0064 - val_loss: 0.0059
Epoch 12/1000
809/809 - 81s - loss: 0.0062 - val_loss: 0.0055
Epoch 13/1000
809/809 - 81s - loss: 0.0060 - val_loss: 0.0059
Epoch 14/1000
809/809 - 81s - loss: 0.0058 - val_loss: 0.0052
Epoch 15/1000
809/809 - 81s - loss: 0.0057 - val_loss: 0.0049
Epoch 16/1000
809/809 - 81s - loss: 0.0055 - val_loss: 0.0

Epoch 76/1000
809/809 - 78s - loss: 0.0030 - val_loss: 0.0030
Epoch 77/1000
809/809 - 78s - loss: 0.0030 - val_loss: 0.0026
Epoch 78/1000
809/809 - 78s - loss: 0.0030 - val_loss: 0.0026
Epoch 79/1000
809/809 - 78s - loss: 0.0030 - val_loss: 0.0026
Epoch 80/1000
809/809 - 78s - loss: 0.0029 - val_loss: 0.0024
Epoch 81/1000
809/809 - 78s - loss: 0.0029 - val_loss: 0.0025
Epoch 82/1000
809/809 - 78s - loss: 0.0029 - val_loss: 0.0024
注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 92s - loss: 0.0766 - val_loss: 0.0246
Epoch 2/1000
809/809 - 78s - loss: 0.0163 - val_loss: 0.0109
Epoch 3/1000
809/809 - 78s - loss: 0.0096 - val_loss: 0.0080
Epoch 4/1000
809/809 - 78s - loss: 0.0079 - val_loss: 0.0072
Epoch 5/1000
809/809 - 78s - loss: 0.0072 - val_loss: 0.0064
Epoch 6/1000
809/809 - 78s - loss: 0.0068 - val_loss: 0.0061
Epoch 7/1000
809/809 - 78s - loss: 0.0065 - val_loss: 0.0057
Epoch 8/1000
809/809 - 78s - loss: 0.0062 - val_loss: 0.0059
Epoch 9/1000
809/809 - 78s - loss: 0.0060 - val_loss: 0.00

Epoch 59/1000
809/809 - 80s - loss: 0.0040 - val_loss: 0.0035
Epoch 60/1000
809/809 - 80s - loss: 0.0040 - val_loss: 0.0034
Epoch 61/1000
809/809 - 80s - loss: 0.0040 - val_loss: 0.0036
Epoch 62/1000
809/809 - 80s - loss: 0.0040 - val_loss: 0.0037
Epoch 63/1000
809/809 - 80s - loss: 0.0040 - val_loss: 0.0040
Epoch 64/1000
809/809 - 79s - loss: 0.0040 - val_loss: 0.0033
Epoch 65/1000
809/809 - 80s - loss: 0.0039 - val_loss: 0.0033
Epoch 66/1000
809/809 - 80s - loss: 0.0039 - val_loss: 0.0036
Epoch 67/1000
809/809 - 79s - loss: 0.0039 - val_loss: 0.0034
Epoch 68/1000
809/809 - 80s - loss: 0.0039 - val_loss: 0.0038
Epoch 69/1000
809/809 - 80s - loss: 0.0039 - val_loss: 0.0032
Epoch 70/1000
809/809 - 80s - loss: 0.0039 - val_loss: 0.0036
Epoch 71/1000
809/809 - 79s - loss: 0.0038 - val_loss: 0.0035
Epoch 72/1000
809/809 - 79s - loss: 0.0038 - val_loss: 0.0038
Epoch 73/1000
809/809 - 79s - loss: 0.0038 - val_loss: 0.0035
Epoch 74/1000
809/809 - 79s - loss: 0.0038 - val_loss: 0.0033
Epoch 75

### 30-150m

In [34]:
#depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
depth = [30,50,100,125,150]
for i in range(0,5): 
    input_cnn_train = np.concatenate((x_cnn_train_split[:,:,:,:5],x_cnn_train_split[:,:,:,i+5:i+6]),axis=-1) 
    input_lstm_train = np.concatenate((x_lstm_train_split[:,:,:5],x_lstm_train_split[:,:,i+5:i+6]),axis=-1)
    input_cnn_val = np.concatenate((x_cnn_val[:,:,:,:5],x_cnn_val[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_val = np.concatenate((x_lstm_val[:,:,:5],x_lstm_val[:,:,i+5:i+6]),axis=-1)  
    
    train_model(input_cnn_train,input_lstm_train,y_train_split[:,i:i+1],
                input_cnn_val,input_lstm_val,y_val[:,i:i+1],
                '../model/cnn_lstm_jc_newVariable/anom/cnn_lstm_jc_model_newVariable_'+str(depth[i])+'m.h5')

注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 112s - loss: 0.1836 - val_loss: 0.1105
Epoch 2/1000
809/809 - 81s - loss: 0.0958 - val_loss: 0.0824
Epoch 3/1000
809/809 - 81s - loss: 0.0788 - val_loss: 0.0702
Epoch 4/1000
809/809 - 82s - loss: 0.0693 - val_loss: 0.0633
Epoch 5/1000
809/809 - 82s - loss: 0.0639 - val_loss: 0.0603
Epoch 6/1000
809/809 - 82s - loss: 0.0588 - val_loss: 0.0527
Epoch 7/1000
809/809 - 82s - loss: 0.0552 - val_loss: 0.0490
Epoch 8/1000
809/809 - 82s - loss: 0.0525 - val_loss: 0.0471
Epoch 9/1000
809/809 - 82s - loss: 0.0493 - val_loss: 0.0427
Epoch 10/1000
809/809 - 82s - loss: 0.0471 - val_loss: 0.0433
Epoch 11/1000
809/809 - 82s - loss: 0.0451 - val_loss: 0.0420
Epoch 12/1000
809/809 - 82s - loss: 0.0433 - val_loss: 0.0405
Epoch 13/1000
809/809 - 82s - loss: 0.0415 - val_loss: 0.0370
Epoch 14/1000
809/809 - 82s - loss: 0.0401 - val_loss: 0.0376
Epoch 15/1000
809/809 - 82s - loss: 0.0390 - val_loss: 0.0330
Epoch 16/1000
809/809 - 82s - loss: 0.0376 - val_loss: 0.0

注意力被执行了一次
注意力被执行了一次
Epoch 1/1000
809/809 - 95s - loss: 0.2911 - val_loss: 0.1984
Epoch 2/1000
809/809 - 79s - loss: 0.1723 - val_loss: 0.1572
Epoch 3/1000
809/809 - 80s - loss: 0.1438 - val_loss: 0.1281
Epoch 4/1000
809/809 - 80s - loss: 0.1278 - val_loss: 0.1196
Epoch 5/1000
809/809 - 80s - loss: 0.1165 - val_loss: 0.1034
Epoch 6/1000
809/809 - 80s - loss: 0.1088 - val_loss: 0.1001
Epoch 7/1000
809/809 - 80s - loss: 0.1022 - val_loss: 0.0967
Epoch 8/1000
809/809 - 80s - loss: 0.0966 - val_loss: 0.1049
Epoch 9/1000
809/809 - 80s - loss: 0.0920 - val_loss: 0.0878
Epoch 10/1000
809/809 - 80s - loss: 0.0877 - val_loss: 0.0796
Epoch 11/1000
809/809 - 80s - loss: 0.0841 - val_loss: 0.0763
Epoch 12/1000
809/809 - 80s - loss: 0.0809 - val_loss: 0.0790
Epoch 13/1000
809/809 - 80s - loss: 0.0784 - val_loss: 0.0768
Epoch 14/1000
809/809 - 80s - loss: 0.0757 - val_loss: 0.0665
Epoch 15/1000
809/809 - 80s - loss: 0.0734 - val_loss: 0.0735
Epoch 16/1000
809/809 - 80s - loss: 0.0714 - val_loss: 0.06

Epoch 133/1000
809/809 - 80s - loss: 0.0327 - val_loss: 0.0302
Epoch 134/1000
809/809 - 80s - loss: 0.0326 - val_loss: 0.0315
Epoch 135/1000
809/809 - 80s - loss: 0.0326 - val_loss: 0.0287
Epoch 136/1000
809/809 - 80s - loss: 0.0325 - val_loss: 0.0281
Epoch 137/1000
809/809 - 80s - loss: 0.0325 - val_loss: 0.0266
Epoch 138/1000
809/809 - 80s - loss: 0.0324 - val_loss: 0.0275
Epoch 139/1000
809/809 - 80s - loss: 0.0325 - val_loss: 0.0268
Epoch 140/1000
809/809 - 80s - loss: 0.0324 - val_loss: 0.0277
Epoch 141/1000
809/809 - 80s - loss: 0.0322 - val_loss: 0.0269
Epoch 142/1000
809/809 - 80s - loss: 0.0323 - val_loss: 0.0260
Epoch 143/1000
809/809 - 80s - loss: 0.0322 - val_loss: 0.0271
Epoch 144/1000
809/809 - 80s - loss: 0.0321 - val_loss: 0.0267
Epoch 145/1000
809/809 - 80s - loss: 0.0321 - val_loss: 0.0265
Epoch 146/1000
809/809 - 80s - loss: 0.0320 - val_loss: 0.0280
Epoch 147/1000
809/809 - 80s - loss: 0.0320 - val_loss: 0.0276
Epoch 148/1000
809/809 - 80s - loss: 0.0319 - val_loss:

Epoch 111/1000
809/809 - 78s - loss: 0.0532 - val_loss: 0.0428
Epoch 112/1000
809/809 - 78s - loss: 0.0530 - val_loss: 0.0426
Epoch 113/1000
809/809 - 78s - loss: 0.0530 - val_loss: 0.0445
Epoch 114/1000
809/809 - 78s - loss: 0.0528 - val_loss: 0.0419
Epoch 115/1000
809/809 - 78s - loss: 0.0526 - val_loss: 0.0468
Epoch 116/1000
809/809 - 78s - loss: 0.0527 - val_loss: 0.0439
Epoch 117/1000
809/809 - 78s - loss: 0.0525 - val_loss: 0.0430
Epoch 118/1000
809/809 - 78s - loss: 0.0523 - val_loss: 0.0485
Epoch 119/1000
809/809 - 78s - loss: 0.0523 - val_loss: 0.0444
Epoch 120/1000
809/809 - 78s - loss: 0.0522 - val_loss: 0.0481
Epoch 121/1000


KeyboardInterrupt: 

# 模型评估

In [35]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score:", r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [37]:
#look_loss(history)

## 对指定层进行评估

In [23]:
input_cnn_test = np.concatenate((x_cnn_test[:,:,:,:5],x_cnn_test[:,:,:,9:]),axis=-1)  
input_lstm_test = np.concatenate((x_lstm_test[:,:,:5],x_lstm_test[:,:,9:]),axis=-1)  

In [24]:
y_test.shape

(1278144, 15)

In [30]:
best_model = tf.keras.models.load_model('../model/cnn_lstm_jc_model_newVariable3_1000m.h5')                                      
all_estimate(best_model,[input_cnn_test,input_lstm_test],y_test[:,14:])

Test RMSE: 0.146
acc: 99.04137843484156
R² score: 0.9783923597387845


## 对所有层进行评估

In [30]:
gc.collect()

0

In [38]:
# 模型预测30-150m
depth = [30,50,100,125,150]
for i in range(0,2):
    best_model = tf.keras.models.load_model('../model/cnn_lstm_jc_newVariable/anom/cnn_lstm_jc_model_newVariable_'+str(depth[i])+'m.h5')                                       
    print(str(depth[i])+'m温度异常：测试集：')
    input_cnn_test = np.concatenate((x_cnn_test[:,:,:,:5],x_cnn_test[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_test = np.concatenate((x_lstm_test[:,:,:5],x_lstm_test[:,:,i+5:i+6]),axis=-1)  
    #all_estimate(best_model,x_test,y_test[:,i:i+1])
    all_estimate(best_model,[input_cnn_test,input_lstm_test],y_test[:,i:i+1])

30m温度异常：测试集：
Test RMSE: 0.308
acc: 95.29316325196781
R² score: 0.9055007658567773
50m温度异常：测试集：
Test RMSE: 0.415
acc: 91.11582825096627
R² score: 0.8301545888386728


In [ ]:
Y_true = np.load("../data_test/xy/Y_30_150m.npy")

In [ ]:
# 模型预测200 - 500
depth =[200,250,300,400,500]
for i in range(0,5):
    best_model = tf.keras.models.load_model('../model/cnn_lstm_jc_newVariable/cnn_lstm_jc_model_newVariable_end_'+str(depth[i])+'m.h5')                                       
    print(str(depth[i])+'m温度：测试集：')
    input_cnn_test = np.concatenate((x_cnn_test[:,:,:,:5],x_cnn_test[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_test = np.concatenate((x_lstm_test[:,:,:5],x_lstm_test[:,:,i+5:i+6]),axis=-1)  
    #all_estimate(best_model,x_test,y_test[:,i:i+1])
    all_estimate(best_model,[input_cnn_test,input_lstm_test],y_test[:,i+5:i+6])

200m温度：测试集：
Test RMSE: 0.117
acc: 99.39993303620793
R² score: 0.9865203293157211
250m温度：测试集：
Test RMSE: 0.106
acc: 99.49293516745465
R² score: 0.9886119740543015
300m温度：测试集：
Test RMSE: 0.105
acc: 99.49908900642988
R² score: 0.9887504195253631
400m温度：测试集：
Test RMSE: 0.121
acc: 99.3497877332884
R² score: 0.9844467380580297
500m温度：测试集：


In [23]:
# 模型预测600 - 1000
depth =[600, 700, 800, 900, 1000]
for i in range(0,5):
    best_model = tf.keras.models.load_model('../model/cnn_lstm_jc_newVariable/cnn_lstm_jc_model_newVariable_end_'+str(depth[i])+'m.h5')                                       
    print(str(depth[i])+'m温度：测试集：')
    input_cnn_test = np.concatenate((x_cnn_test[:,:,:,:5],x_cnn_test[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_test = np.concatenate((x_lstm_test[:,:,:5],x_lstm_test[:,:,i+5:i+6]),axis=-1)  
    #all_estimate(best_model,x_test,y_test[:,i:i+1])
    all_estimate(best_model,[input_cnn_test,input_lstm_test],y_test[:,i+10:i+11])

600m温度：测试集：
Test RMSE: 0.158
acc: 98.81586119523365
R² score: 0.9733420890594672
700m温度：测试集：
Test RMSE: 0.158
acc: 98.8341717224862
R² score: 0.9737155012463298
800m温度：测试集：
Test RMSE: 0.144
acc: 99.01211432084158
R² score: 0.9785477221984661
900m温度：测试集：
Test RMSE: 0.147
acc: 99.04171927215309
R² score: 0.9779162080885253
1000m温度：测试集：
Test RMSE: 0.144
acc: 99.04718753622689
R² score: 0.9787711702624015


# 模型预测结果保存

## 获取所有层的预测结果

In [24]:
# 模型预测
depth = [30,50,100,125,150]
result = [] 
for i in range(0,5):
    best_model = tf.keras.models.load_model('../model/cnn_lstm_jc_model_newVariable3_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m的温度')
    input_cnn_test = np.concatenate((x_cnn_test[:,:,:,:5],x_cnn_test[:,:,:,i+5:i+6]),axis=-1)  
    input_lstm_test = np.concatenate((x_lstm_test[:,:,:5],x_lstm_test[:,:,i+5:i+6]),axis=-1) 
    testPred = best_model.predict([input_cnn_test,input_lstm_test])
    result.append(testPred)

预测第30m的温度
预测第50m的温度
预测第100m的温度
预测第125m的温度
预测第150m的温度


## 对预测结果进行反归一化

### 保存预测结果

In [26]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,5)
print(result.shape)

(5, 1278144, 1)
(1278144, 1, 5)
(1278144, 5)


In [8]:
#np.save("../data_test/result/result_30_150.npy",result)

# result_30_150 = np.load("../data_test/result/result_30_150.npy")
# result_200_500 = np.load("../data_test/result/result_200_500.npy")
# result_600_1000 = np.load("../data_test/result/result_600_1000.npy")

In [20]:
result_30_1000 = np.hstack((result_30_150, result_200_500, result_600_1000))

In [18]:
#np.save("../data_test/result/result_30_1000.npy",result_30_1000)
result_30_1000 = np.load("../data_test/result/result_30_1000.npy")

### 反归一化

In [19]:
result_unscaled = scaler_l.inverse_transform(result_30_1000)
y_test_unscaled = scaler_l.inverse_transform(y_test)

In [20]:
y_test_unscaled.shape,result_unscaled.shape

((1278144, 15), (1278144, 15))

### 评估反归一化之后的结果

In [48]:
depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
for i in range(0,15):
    mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
    print(str(depth[i])+'m Test mse: %.4f' % mse)

30m Test mse: 0.1894
50m Test mse: 0.3868
100m Test mse: 0.4537
125m Test mse: 0.4602
150m Test mse: 0.4742
200m Test mse: 0.3267
250m Test mse: 0.1919
300m Test mse: 0.1473
400m Test mse: 0.1230
500m Test mse: 0.0901
600m Test mse: 0.0596
700m Test mse: 0.0334
800m Test mse: 0.0179
900m Test mse: 0.0131
1000m Test mse: 0.0094


In [50]:
for i in range(0,15):
    mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
    rmse = np.sqrt(mse)
    print(str(depth[i])+'m Test rmse: %.4f' % rmse)

30m Test rmse: 0.4352
50m Test rmse: 0.6220
100m Test rmse: 0.6736
125m Test rmse: 0.6784
150m Test rmse: 0.6886
200m Test rmse: 0.5716
250m Test rmse: 0.4381
300m Test rmse: 0.3838
400m Test rmse: 0.3508
500m Test rmse: 0.3001
600m Test rmse: 0.2442
700m Test rmse: 0.1827
800m Test rmse: 0.1338
900m Test rmse: 0.1146
1000m Test rmse: 0.0970


### 计算相对误差

In [21]:
def relative_error(result_unscaled, y_test_unscaled):   # 反演值   真实值
    # 处理真实值为零的情况
    y_test_unscaled_abs = np.abs(y_test_unscaled)
    
    # 计算相对误差
    absolute_errors = np.abs(result_unscaled - y_test_unscaled) #\反演值 - 真实值\
    
    mask = y_test_unscaled_abs>0.1
    
    relative_errors = np.zeros_like(absolute_errors)
    relative_errors[mask] = (absolute_errors[mask] / y_test_unscaled_abs[mask]) * 100
    
    #relative_errors = (absolute_errors / y_test_unscaled_abs) * 100
    print(relative_errors.shape)
    print(relative_errors[mask].shape)
    return relative_errors[mask]

In [22]:
depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]
for i in range(0,15):
    a = relative_error(result_unscaled[:,i],y_test_unscaled[:,i])
    print(f"相对误差数组为:",np.mean(a))

(1278144,)
(1277699,)
相对误差数组为: 2.0158266878960776
(1278144,)
(1277394,)
相对误差数组为: 3.2796243981492186
(1278144,)
(1277309,)
相对误差数组为: 4.232329226864575
(1278144,)
(1277245,)
相对误差数组为: 4.498242758347381
(1278144,)
(1277614,)
相对误差数组为: 4.257922895630924
(1278144,)
(1277551,)
相对误差数组为: 3.978629649399007
(1278144,)
(1277664,)
相对误差数组为: 3.461387209941452
(1278144,)
(1277951,)
相对误差数组为: 3.0403223027609427
(1278144,)
(1278134,)
相对误差数组为: 2.9840189428768333
(1278144,)
(1278144,)
相对误差数组为: 2.828082263391153
(1278144,)
(1278144,)
相对误差数组为: 2.636054132857473
(1278144,)
(1278144,)
相对误差数组为: 2.2715606109777675
(1278144,)
(1278144,)
相对误差数组为: 2.015759374065442
(1278144,)
(1278144,)
相对误差数组为: 1.9648972223668462
(1278144,)
(1278144,)
相对误差数组为: 1.790255424188912


### 按照月份来计算R^2 和RMSE 

In [76]:
y_test_unscaled.shape,result_unscaled.shape 

((1278144, 15), (1278144, 15))

In [86]:
result_unscaled_month  = result_unscaled.reshape(12,-1,15)
y_test_unscaled_month  = y_test_unscaled.reshape(12,-1,15)

In [87]:
result_unscaled_month.shape 

(12, 106512, 15)

In [109]:
def respective_estimate(month,result,y_test):
    re2_list = []
    rmse_list = []
    for i in range(15):  # 15代表15个深度级
        # 模型预测
        testPred =result[(month-1),:,i:i+1]
        a = y_test[(month-1),:,i:i+1]
        r2 = r2_score(a,testPred)
        rmse = sqrt(mean_squared_error(a,testPred))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [131]:
# 4月的R² 和 RMSE
respective_estimate(4,result_unscaled_month,y_test_unscaled_month)

([0.9983942257968306,
  0.9952239259620208,
  0.990665471427948,
  0.990545579448546,
  0.9837646654456049,
  0.9837520202175121,
  0.987363462206706,
  0.9862904154851098,
  0.9811371726327283,
  0.9735540295997779,
  0.9682056570094634,
  0.9651267528269689,
  0.9750055546788515,
  0.9748213589918855,
  0.9777062749987364],
 [0.35006147014310945,
  0.5891170879127386,
  0.7605341981134914,
  0.7100527725700463,
  0.8283546191177457,
  0.6400132124641215,
  0.46626538933321404,
  0.4277273616675807,
  0.38983938467727985,
  0.3442235549662269,
  0.26997391646310714,
  0.21091643334477014,
  0.1442357556628954,
  0.12209441988016205,
  0.09887022482908689])

### 计算不同深度的温度变化的平均均方根（RMS）值

In [198]:
# 计算每一年的平均值
months_per_year = 12
num_years = 11
data_shape = (132, 250, 610, 15)
averaged_data = np.mean(temp_depth_use.reshape(num_years, months_per_year, *data_shape[1:]), axis=1)

In [201]:
averaged_data.shape 

(11, 250, 610, 15)

In [200]:
# 计算每个深度的年际温度变化
yearly_changes_2 = np.diff(averaged_data, axis=0)
print(yearly_changes_2.shape)
yearly_changes_2 = yearly_changes_2.reshape(-1,15)
print(yearly_changes_2.shape)
rms_per_depth_2 = np.sqrt(np.nanmean(np.square(yearly_changes_2), axis=0))
print("不同深度的温度年际变化的平均 RMS:", rms_per_depth_2)

(10, 250, 610, 15)
(1525000, 15)
不同深度的温度年际变化的平均 RMS: [0.62744355 0.8121952  1.0274146  0.9404541  0.8155954  0.5527513
 0.4246711  0.37204105 0.33037296 0.28008792 0.20921904 0.14691857
 0.10419048 0.08102585 0.06601328]


### 计算不同深度水平的垂直温度梯度

In [93]:
depth = [30,50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [112]:
# 计算深度差
delta_depth = np.diff(depth)  # 深度差

# 扩展 delta_depth 以匹配温度差的维度
delta_depth_expanded = delta_depth[np.newaxis, np.newaxis, np.newaxis, :]  #(1, 1, 1, 14)

# 计算每个月份的温度差
delta_temperature = np.diff(temp_depth_use, axis=-1)  # 在深度维度上计算差值

# 计算每个月份的垂直温度梯度
vertical_temperature_gradient_per_month = delta_temperature / delta_depth_expanded

# 计算所有月份的平均垂直温度梯度
average_vertical_temperature_gradient = np.nanmean(vertical_temperature_gradient_per_month, axis=0)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20292\93721926.py:14: RuntimeWarning: Mean of empty slice
  average_vertical_temperature_gradient = np.nanmean(vertical_temperature_gradient_per_month, axis=0)


In [113]:
print(delta_depth_expanded.shape)
print(delta_temperature.shape)
print(vertical_temperature_gradient_per_month.shape)
print(average_vertical_temperature_gradient.shape)

(1, 1, 1, 14)
(132, 250, 610, 14)
(132, 250, 610, 14)
(250, 610, 14)


In [114]:
average_vertical_temperature_gradient  = average_vertical_temperature_gradient.reshape(-1,14)
average_vertical_temperature_gradient  = np.nanmean(average_vertical_temperature_gradient,axis = 0)
average_vertical_temperature_gradient

array([-0.0540533 , -0.05762655, -0.05383809, -0.05034414, -0.04223369,
       -0.0277066 , -0.01938522, -0.01563386, -0.01306995, -0.0103042 ,
       -0.00751749, -0.00544361, -0.00422482, -0.00350151])

# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [51]:
nan_mask_cat = np.load("../data_test/nan_mask_cat.npy")

In [52]:
nan_mask_cat.shape 

(144000,)

In [53]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full((144000,12*15), np.nan)  
print(testPred_with_nan.shape)
y_test_with_nan = np.full((144000,12*15), np.nan)  
print(y_test_with_nan.shape)

(144000, 180)
(144000, 180)


In [55]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,15)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*15)
print(result_unscaled_reshape.shape)

# ======================================== #
y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,15)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*15)
print(y_test_unscaled_reshape.shape)


(12, 106512, 15)
(106512, 12, 15)
(106512, 180)
(12, 106512, 15)
(106512, 12, 15)
(106512, 180)


In [60]:
num = 0
for i,nan_mask in enumerate(nan_mask_cat):   # i从0开始  在nan_mask_cat中true代表有效值，False代表无效值

    if (nan_mask == True): # true 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
        
print(num)

106512


In [62]:
testPred_with_nan = testPred_with_nan.reshape(240, 600,12,15)   #(144000, 180) --> (240, 600,12,15) 
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))  


y_test_with_nan = y_test_with_nan.reshape(240, 600,12,15)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 15, 240, 600) (12, 15, 240, 600)


## 保存

In [70]:
lat = []
lon = []
time = []
lon.append(data_ssw['lon'][480:1080].data)
lat.append(data_ssh1['lat'][360:600].data)

In [73]:
start_year = 2021
end_year =2021
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [74]:
new_NC = nc.Dataset("result30_1000m_jc.nc", 'w', format='NETCDF4')



new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 15)
new_NC.createDimension('time', len(time))


new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('true_sta', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_sta', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2021-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'


#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['true_sta'][:]=np.array(y_test_with_nan)
new_NC.variables['pred_sta'][:]=np.array(testPred_with_nan)


#最后记得关闭文件
new_NC.close()
